In [126]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import struct
import warnings

from typing import Tuple, Literal, Optional, List, Dict
# Importation des modules nécessaires de scikit-learn
from scipy.stats import entropy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.base import clone
from sklearn.base import ClassifierMixin

# Classificateurs de scikit-learn
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier
)
from sklearn.linear_model import (
    LogisticRegression,
    RidgeClassifier,
    ElasticNetCV
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE


# LightGBM
import lightgbm

# Configuration du logging
import logging
# Ignorer les warnings
warnings.filterwarnings("ignore")


In [3]:
# Supprimer tous les handlers existants
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Configurer le logger avec un format sans date/heure/niveau
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [4]:
def split_dataset(X: np.ndarray, y: np.ndarray, labeled_ratio: float, test_ratio: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Divise un dataset en ensembles d'entraînement, de test et de pool (non labellisé) selon les ratios spécifiés.
    
    Paramètres :
        X (np.ndarray) : Matrice des caractéristiques (features), de forme (n_samples, n_features).
        y (np.ndarray) : Vecteur des labels, de forme (n_samples,).
        labeled_ratio (float) : Proportion d'échantillons dans l'ensemble d'entraînement labellisé.
        test_ratio (float) : Proportion d'échantillons dans l'ensemble de test.
        
    Retourne :
        Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray] :
            - X_train (np.ndarray) : Matrice des caractéristiques de l'ensemble d'entraînement.
            - X_pool (np.ndarray) : Matrice des caractéristiques de l'ensemble de pool (non labellisé).
            - y_train (np.ndarray) : Vecteur des labels de l'ensemble d'entraînement.
            - y_pool (np.ndarray) : Vecteur des labels de l'ensemble de pool (non labellisé).
            - X_test (np.ndarray) : Matrice des caractéristiques de l'ensemble de test.
            - y_test (np.ndarray) : Vecteur des labels de l'ensemble de test.
    """
    
    total_samples = X.shape[0]
    
    # Calcul du nombre d'échantillons labellisés et du nombre d'échantillons de test
    nb_labeled = int(labeled_ratio * total_samples)
    test_size = int(test_ratio * total_samples)
    
    # Diviser le dataset en un ensemble d'entraînement (labellisé) et un ensemble pool (non labellisé)
    X_train, X_pool, y_train, y_pool = train_test_split(X, y, test_size=(1 - labeled_ratio), random_state=42)
    
    # Diviser l'ensemble pool pour obtenir l'ensemble de test
    X_test, y_test = X_pool[:test_size], y_pool[:test_size]
    X_pool, y_pool = X_pool[test_size:], y_pool[test_size:]
    
    # Retourner les ensembles
    return X_train, X_pool, y_train, y_pool, X_test, y_test


In [5]:
def calculate_uncertainty(probabilities: np.ndarray, method: Literal["entropy", "margin", "least_confident", "random"] = "entropy") -> np.ndarray:
    """
    Calcule l'incertitude des prédictions basées sur les probabilités de classe.
    
    Paramètres :
        probabilities (np.ndarray) : Matrice des probabilités de classe pour chaque échantillon, de forme (n_samples, n_classes).
        method (str, optionnel) : Méthode d'incertitude à utiliser. Les options sont :
            - "entropy" : Entropie de Shannon.
            - "margin" : Différence entre les deux plus grandes probabilités.
            - "least_confident" : Complément de la plus grande probabilité.
            - "random" : Valeur aléatoire entre 0 et 1 pour chaque échantillon.
        
    Retourne :
        np.ndarray : Un tableau d'incertitude de forme (n_samples,) avec les scores d'incertitude pour chaque échantillon.
        
    Lève :
        ValueError : Si la méthode d'incertitude spécifiée n'est pas reconnue.
    """
    
    # Calcul de l'incertitude en fonction de la méthode choisie
    if method == "entropy":
        # Entropie de Shannon
        return entropy(probabilities.T, base=2)  # Probabilités doivent être de forme (n_classes, n_samples)
    
    elif method == "margin":
    # Si vous avez seulement deux classes
        if probabilities.shape[1] == 1:
            # Cas où la probabilité de chaque classe est un seul vecteur (probabilité = 1 pour une classe)
            return np.zeros(probabilities.shape[0])  # Pas de marge, car on est complètement confiant (différence = 0)
    
        elif probabilities.shape[1] == 2:
            # Cas classique avec deux classes
            prob_class_1 = probabilities[:, 0]
            prob_class_2 = probabilities[:, 1]
            return np.abs(prob_class_1 - prob_class_2)
    
        else:
            # Cas avec plus de deux classes, utiliser la marge entre les deux plus grandes probabilités
            sorted_probs = np.sort(probabilities, axis=1)
            return sorted_probs[:, -1] - sorted_probs[:, -2]

    
    elif method == "least_confident":
        # Complément de la probabilité la plus élevée (1 - max(probabilities))
        return 1 - np.max(probabilities, axis=1)
    
    elif method == "random":
        # Valeurs aléatoires entre 0 et 1
        return np.random.rand(probabilities.shape[0])
    
    else:
        # Si une méthode d'incertitude non reconnue est fournie, lever une exception
        raise ValueError("Méthode d'incertitude non reconnue")

In [6]:
def train_and_evaluate(X_train: np.ndarray, y_train: np.ndarray, X_test: np.ndarray, y_test: np.ndarray, model_class: type[ClassifierMixin]) -> tuple:
    """
    Entraîne un modèle sur un ensemble d'entraînement et évalue sa performance sur un ensemble de test.
    
    Paramètres :
        X_train (np.ndarray) : Les caractéristiques d'entraînement, de forme (n_samples, n_features).
        y_train (np.ndarray) : Les étiquettes d'entraînement, de forme (n_samples,).
        X_test (np.ndarray) : Les caractéristiques de test, de forme (n_samples, n_features).
        y_test (np.ndarray) : Les étiquettes de test, de forme (n_samples,).
        model_class (type) : La classe du modèle de classification (doit être un sous-type de `ClassifierMixin`, par exemple `RandomForestClassifier`).

    Retourne :
        tuple : Un tuple contenant :
            - Le modèle entraîné (`model`).
            - La précision sur l'ensemble de test (`accuracy`).
    """
    # Initialisation du modèle
    model = model_class()
    
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédiction sur l'ensemble de test
    y_pred = model.predict(X_test)
    
    # Calcul de la précision
    accuracy = accuracy_score(y_test, y_pred)
    
    return model, accuracy

In [7]:
def select_uncertain_samples(model: ClassifierMixin, X_pool: np.ndarray, method: str, batch_size: int) -> np.ndarray:
    """
    Sélectionne les échantillons les plus incertains dans un ensemble de données non labellisées, en fonction de la méthode d'incertitude choisie.
    
    Paramètres :
        model (ClassifierMixin) : Le modèle d'apprentissage supervisé entraîné, avec une méthode `predict_proba` (par exemple, un classifieur comme `RandomForestClassifier` ou `SVC`).
        X_pool (np.ndarray) : Le tableau des caractéristiques des échantillons non labellisés, de forme (n_samples, n_features).
        method (str) : La méthode d'incertitude à utiliser pour calculer l'incertitude. Les options disponibles sont :
            - "entropy"
            - "margin"
            - "least_confident"
            - "random"
        batch_size (int) : Le nombre d'échantillons à sélectionner en fonction de l'incertitude.

    Retourne :
        np.ndarray : Un tableau des indices des `batch_size` échantillons les plus incertains dans `X_pool`.
    """
    # Probabilités prédites pour chaque échantillon
    probabilities = model.predict_proba(X_pool)
    
    # Calcul des incertitudes en fonction de la méthode choisie
    uncertainties = calculate_uncertainty(probabilities, method=method)
    
    # Sélection des indices des échantillons les plus incertains
    if method == "margin":
        # La méthode "margin" prend les échantillons ayant les plus petites marges entre les deux classes les plus probables
        return np.argsort(uncertainties)[:batch_size]
    else:
        # Pour les autres méthodes, on prend les échantillons ayant les incertitudes les plus élevées
        return np.argsort(uncertainties)[-batch_size:]

In [8]:
def update_labeled_unlabeled_sets(X_train: np.ndarray, y_train: np.ndarray, X_pool: np.ndarray, y_pool: np.ndarray, uncertain_indices: np.ndarray) -> tuple:
    """
    Met à jour les ensembles d'échantillons labellisés et non-labellisés après sélection des échantillons incertains.
    
    Paramètres :
        X_train (np.ndarray) : Ensemble des caractéristiques des échantillons labellisés (de taille (n_labeled_samples, n_features)).
        y_train (np.ndarray) : Ensemble des labels des échantillons labellisés (de taille (n_labeled_samples,)).
        X_pool (np.ndarray) : Ensemble des caractéristiques des échantillons non labellisés (de taille (n_pool_samples, n_features)).
        y_pool (np.ndarray) : Ensemble des labels des échantillons non labellisés (de taille (n_pool_samples,)).
        uncertain_indices (np.ndarray) : Indices des échantillons les plus incertains dans `X_pool`, qui doivent être ajoutés à `X_train` (de taille (batch_size,)).

    Retourne :
        tuple : 
            - X_train (np.ndarray) : Ensemble mis à jour des caractéristiques des échantillons labellisés.
            - y_train (np.ndarray) : Ensemble mis à jour des labels des échantillons labellisés.
            - X_pool (np.ndarray) : Ensemble mis à jour des caractéristiques des échantillons non labellisés.
            - y_pool (np.ndarray) : Ensemble mis à jour des labels des échantillons non labellisés.
    """
    # Ajout des échantillons incertains à l'ensemble des échantillons labellisés
    X_train = np.vstack((X_train, X_pool[uncertain_indices]))  # Empile les échantillons incertains sur l'ensemble de train
    y_train = np.hstack((y_train, y_pool[uncertain_indices]))  # Ajoute les labels des échantillons incertains

    # Création d'un masque pour supprimer les échantillons incertains de l'ensemble non-labellisé
    mask = np.ones(len(X_pool), dtype=bool)  # Crée un masque de True pour tous les échantillons non labellisés
    mask[uncertain_indices] = False  # Marque les indices incertains comme False dans le masque
    
    # Mise à jour de l'ensemble non-labellisé en excluant les échantillons incertains
    X_pool = X_pool[mask]  # Ne garde que les échantillons non incertains
    y_pool = y_pool[mask]  # Ne garde que les labels des échantillons non incertains

    return X_train, y_train, X_pool, y_pool

In [9]:
def hybrid_uncertainty(model, X_pool: np.ndarray, w1: float = 0.33, w2: float = 0.33, w3: float = 0.34) -> np.ndarray:
    """
    Combine plusieurs mesures d'incertitude (confiance minimale, marge et entropie) en une seule mesure pondérée.
    
    Args:
        model (sklearn.base.BaseEstimator): Le modèle de classification entraîné.
        X_pool (np.ndarray): Les échantillons non labellisés (de taille (n_pool_samples, n_features)).
        w1 (float, optional): Poids de la mesure "least confident" (par défaut 0.33).
        w2 (float, optional): Poids de la mesure "margin" (par défaut 0.33).
        w3 (float, optional): Poids de la mesure "entropy" (par défaut 0.34).
    
    Returns:
        np.ndarray: Un tableau des scores d'incertitude combinés (de taille (n_pool_samples,)).
    """
    # Prédictions des probabilités pour chaque classe
    proba = model.predict_proba(X_pool)
    
    # Mesure d'incertitude "Least Confident" : 1 - probabilité maximale
    least_confident = 1 - np.max(proba, axis=1)
    
    # Mesure "Margin" : différence entre les deux plus grandes probabilités
    if proba.shape[1] == 1:
        # Si on a qu'une seule classe (par exemple, probabilité = 1 pour une seule classe), on ne peut pas calculer une marge
        margin = np.zeros(proba.shape[0])  # Pas de marge dans ce cas
    else:
        sorted_proba = np.sort(proba, axis=1)
        margin = sorted_proba[:, -1] - sorted_proba[:, -2]
    
    # Entropie des probabilités : mesure de l'incertitude globale
    epsilon = 1e-10  # Ajout d'un petit epsilon pour éviter log(0)
    entropy = -np.sum(proba * np.log(proba + epsilon), axis=1)
    
    # Combinaison des scores d'incertitude avec les poids
    combined_score = w1 * least_confident + w2 * (1 - margin) + w3 * entropy
    
    return combined_score


In [10]:
def select_uncertain_samples_hybrid(model, X_pool: np.ndarray, batch_size: int) -> np.ndarray:
    """
    Sélectionne les échantillons les plus incertains selon la stratégie hybride en combinant 
    plusieurs mesures d'incertitude.
    
    Args:
        model (sklearn.base.BaseEstimator): Le modèle de classification entraîné.
        X_pool (np.ndarray): Les échantillons non labellisés (de taille (n_pool_samples, n_features)).
        batch_size (int): Nombre d'échantillons à sélectionner pour chaque itération.
    
    Returns:
        np.ndarray: Indices des échantillons les plus incertains dans X_pool (de taille (batch_size,)).
    """
    # Calcul des scores d'incertitude combinés (stratégie hybride)
    scores = hybrid_uncertainty(model, X_pool)
    
    # Sélectionner les indices des échantillons les plus incertains
    return np.argsort(scores)[-batch_size:]  # Retourne les indices des scores d'incertitude les plus élevés

In [11]:
def select_uncertain_samples_qbc(method: str, models: list, X_train: np.ndarray, y_train: np.ndarray, 
                                 X_pool: np.ndarray, batch_size: int) -> np.ndarray:
    """
    Sélectionne les échantillons les plus incertains en utilisant le Query by Committee (QBC).
    
    Le QBC évalue l'incertitude des échantillons dans l'ensemble non-labellisé en comparant les 
    prédictions des modèles d'un comité. Trois méthodes sont supportées :
    - "qbc-variance" : variance des prédictions des modèles sur chaque échantillon.
    - "qbc-entropy" : entropie des votes du comité.
    - "qbc-KL" : divergence Kullback-Leibler entre les prédictions des modèles et la probabilité moyenne.
    
    Args:
        method (str): Méthode d'incertitude à utiliser. Une des options suivantes :
                      - "qbc-variance"
                      - "qbc-entropy"
                      - "qbc-KL"
        models (list): Liste des modèles (du comité), chacun devant avoir la méthode `predict_proba`.
        X_train (np.ndarray): Ensemble de données d'entraînement labellisées (n_samples, n_features).
        y_train (np.ndarray): Labels des données d'entraînement (n_samples,).
        X_pool (np.ndarray): Données non-labellisées (n_pool_samples, n_features).
        batch_size (int): Nombre d'échantillons à sélectionner dans chaque itération.
    
    Returns:
        np.ndarray: Indices des échantillons les plus incertains dans `X_pool` (taille = batch_size).
    """
    # Entraîner chaque modèle du comité sur les données labellisées
    for model in models:
        model.fit(X_train, y_train)

    # Obtenir les prédictions de chaque modèle sur le pool de données non labellisées
    predictions = np.array([model.predict_proba(X_pool) for model in models])  # Shape: (n_models, n_samples, n_classes)

    # Initialiser l'incertitude
    uncertainty = None

    if method == "qbc-variance":
        # Calcul de la variance des prédictions pour chaque échantillon
        uncertainty = np.var(predictions, axis=0)  # Variance des probabilités
        uncertainty = np.mean(uncertainty, axis=1)  # Moyenne de la variance sur les classes

    elif method == "qbc-entropy":
        # Comptage des votes pour chaque classe
        n_models, n_samples, n_classes = predictions.shape
        vote_counts = np.zeros((n_samples, n_classes))

        for i in range(n_samples):
            for j in range(n_models):
                # Trouver la classe prédite par chaque modèle
                predicted_class = np.argmax(predictions[j, i, :])
                vote_counts[i, predicted_class] += 1

        # Calcul des probabilités de vote
        vote_probs = vote_counts / n_models

        # Calcul de l'entropie des votes pour chaque échantillon
        uncertainty = np.array([entropy(vote_probs[i, :], base=2) for i in range(n_samples)])

    elif method == "qbc-KL":
        # Calcul de la probabilité moyenne (P_avg) pour chaque échantillon
        P_avg = np.mean(predictions, axis=0)  # Moyenne des prédictions des modèles (n_samples, n_classes)

        # Calcul de la divergence Kullback-Leibler (KL) pour chaque échantillon
        n_samples, n_classes = P_avg.shape
        uncertainty = np.zeros(n_samples)

        for i in range(n_samples):
            for model_preds in predictions[:, i, :]:
                # Calcul de la divergence KL pour chaque échantillon
                uncertainty[i] += np.sum(model_preds * np.log(model_preds / P_avg[i, :]))

    else:
        raise ValueError("Méthode d'incertitude non reconnue. Choisissez parmi 'qbc-variance', 'qbc-entropy' ou 'qbc-KL'.")

    # Sélectionner les indices des échantillons ayant l'incertitude la plus élevée
    uncertain_indices = np.argsort(uncertainty)[-batch_size:]

    return uncertain_indices

In [12]:
#à terminer
def select_uncertain_samples_egl(model, X_pool: np.ndarray, batch_size: int) -> np.ndarray:
    """
    Sélectionne les échantillons les plus informatifs en utilisant la méthode EGL (Expected Gradient Length).
    
    Cette méthode sélectionne les échantillons qui entraîneraient la plus grande mise à jour du modèle
    en fonction de la norme du gradient de la fonction de coût par rapport à chaque échantillon.
    
    Args:
        model : Estimation du modèle de classification (doit être basé sur des gradients, ex : régression logistique).
        X_pool : Données non-labellisées pour lesquelles nous souhaitons calculer l'incertitude.
        batch_size : Nombre d'échantillons à sélectionner.
    
    Returns:
        np.ndarray : Indices des échantillons les plus incertains dans `X_pool`.
    """
    if not hasattr(model, "coef_"):
        raise ValueError("Le modèle doit être une régression logistique ou un autre modèle basé sur des gradients.")
    
    gradients = []
    
    # Obtenir les probabilités de prédiction
    probs = model.predict_proba(X_pool)  # Probabilités pour chaque classe (n_samples, n_classes)
    
    # Calculer l'incertitude pour chaque échantillon en utilisant la méthode EGL
    for i, x in enumerate(X_pool):
        expected_grad = 0
        
        for class_idx in range(probs.shape[1]):  # Parcourir chaque classe
            prob = probs[i, class_idx]
            
            # Créer un vecteur one-hot pour la classe cible
            y_dummy = np.zeros((1, probs.shape[1]))
            y_dummy[0, class_idx] = 1
            
            try:
                # Calcul du gradient pour cette classe en ajustant temporairement le modèle
                model.fit(x.reshape(1, -1), y_dummy.argmax(axis=1))  # Entraînement temporaire sur l'exemple (1, -1) pour l'ajustement de la forme
                grad = np.linalg.norm(model.coef_)  # Norme du gradient
                expected_grad += prob * grad  # Calcul de l'espérance du gradient
            except ValueError:
                # Si une erreur survient, on passe à la classe suivante
                continue
        
        gradients.append(expected_grad)
    
    # Sélection des indices des plus grandes valeurs de gradient
    uncertain_indices = np.argsort(gradients)[-batch_size:]
    
    return uncertain_indices


In [13]:
def compute_information_density(X_pool: np.ndarray, similarity_metric: str = 'cosine') -> np.ndarray:
    """
    Calcule la densité d'information pour chaque point du pool en fonction de la similarité avec les autres points.
    
    Args:
        X_pool (numpy.ndarray): Ensemble non labellisé de taille (n_samples, n_features).
        similarity_metric (str): Type de mesure de similarité ('cosine' ou 'euclidean').
    
    Returns:
        numpy.ndarray: Score de densité pour chaque instance.
    """
    # Calcul de la matrice de similarité en fonction de la métrique choisie
    if similarity_metric == 'cosine':
        similarity_matrix = cosine_similarity(X_pool)
    elif similarity_metric == 'euclidean':
        distance_matrix = np.linalg.norm(X_pool[:, np.newaxis] - X_pool, axis=2)
        similarity_matrix = 1 / (1 + distance_matrix)  # Conversion en similarité
    else:
        raise ValueError("Metric non supportée. Utilisez 'cosine' ou 'euclidean'.")
    
    # Calcul de la moyenne des similarités pour chaque point (densité)
    # Evite l'auto-similarité (similarité d'un point avec lui-même) en mettant la diagonale à zéro
    np.fill_diagonal(similarity_matrix, 0)
    
    density_scores = np.mean(similarity_matrix, axis=1)
    
    # Normalisation des scores de densité entre 0 et 1 (facultatif)
    density_scores /= np.max(density_scores)
    
    return density_scores

In [14]:
def select_uncertain_samples_density(model, X_pool: np.ndarray, batch_size: int, similarity_metric: str = 'cosine') -> list:
    """
    Sélectionne les échantillons en combinant incertitude et densité d'information.
    
    Args:
        model : Modèle de classification entraîné utilisé pour l'incertitude.
        X_pool : Ensemble non labellisé de taille (n_samples, n_features).
        batch_size : Nombre d'échantillons à sélectionner.
        similarity_metric : Type de mesure de similarité ('cosine' ou 'euclidean').
    
    Returns:
        list: Indices des échantillons sélectionnés.
    """
    # 1. Vérification de la méthode de calcul des probabilités pour l'incertitude
    if not hasattr(model, "predict_proba"):
        raise ValueError("Le modèle doit implémenter 'predict_proba' pour calculer l'incertitude.")
    
    probas = model.predict_proba(X_pool)
    # Calcul de l'incertitude (exemple : entropie prédictive)
    uncertainty = -np.sum(probas * np.log(probas + 1e-10), axis=1)  # Évite log(0)
    
    # 2. Calcul de la densité d'information
    density_scores = compute_information_density(X_pool, similarity_metric)
    
    # 3. Combinaison des scores : pondération entre incertitude et densité
    combined_scores = uncertainty * density_scores
    
    # 4. Sélection des indices des échantillons les plus incertains et informatifs
    selected_indices = np.argsort(combined_scores)[-batch_size:]
    
    return selected_indices.tolist()

In [15]:
def select_uncertain_samples_general(method: str, model: object, X_pool: np.ndarray, batch_size: int, models: Optional[List[object]] = None, X_train: Optional[np.ndarray] = None, y_train: Optional[np.ndarray] = None, similarity_metric: str = 'cosine') -> np.ndarray:
    """
    Sélectionne les échantillons les plus incertains selon la méthode spécifiée.
    
    Args:
        method (str): Méthode de sélection. Doit être l'une des suivantes : 
            'qbc-variance', 'qbc-entropy', 'qbc-KL', 'hybrid', 'EGL', 'least_confident', 
            'margin', 'entropy', 'random', 'density'.
        model (object): Modèle de classification entraîné (ex. RandomForestClassifier, LogisticRegression, etc.).
        X_pool (numpy.ndarray): Ensemble non labellisé de taille (n_samples, n_features).
        batch_size (int): Nombre d'échantillons à sélectionner.
        models (list, optionnel): Liste de modèles pour la méthode QBC. Requis pour 'qbc-*'.
        X_train (numpy.ndarray, optionnel): Données d'entraînement labellisées. Requis pour QBC.
        y_train (numpy.ndarray, optionnel): Labels d'entraînement. Requis pour QBC.
        similarity_metric (str, optionnel): Métrique de similarité pour 'density'. Valeurs possibles : 'cosine', 'euclidean'.
    
    Returns:
        numpy.ndarray: Indices des échantillons les plus incertains sélectionnés.
    
    Raises:
        ValueError: Si la méthode est inconnue ou si les arguments nécessaires sont manquants.
    """
    
    # Vérification des arguments nécessaires pour certaines méthodes
    if method in ["qbc-variance", "qbc-entropy", "qbc-KL"]:
        if models is None or X_train is None or y_train is None:
            raise ValueError("Les méthodes QBC nécessitent un comité de modèles et des données d'entraînement labellisées.")
        return select_uncertain_samples_qbc(method, models, X_train, y_train, X_pool, batch_size)
    
    elif method == "hybrid":
        return select_uncertain_samples_hybrid(model, X_pool, batch_size)
    
    elif method == "EGL":
        return select_uncertain_samples_egl(model, X_pool, batch_size)
    
    elif method in ["least_confident", "margin", "entropy"]:
        return select_uncertain_samples(model, X_pool, method, batch_size)
    
    elif method == "random":
        # Sélection aléatoire d'échantillons sans remplacement
        return np.random.choice(len(X_pool), size=batch_size, replace=False)
    
    elif method == "density":
        return select_uncertain_samples_density(model, X_pool, batch_size, similarity_metric)
    
    else:
        raise ValueError(f"Méthode de sélection non reconnue : {method}")


In [16]:
def plot_active_learning_results(dataset_name: str, methods: List[str], accuracies: Dict[str, Dict[str, List[float]]],
                                 labeled_ratio: float, batch_ratio: float, n_iterations: int, 
                                 full_training_accuracies: Dict[str, float], final_accuracies: Dict[str, Dict[str, float]], 
                                 initial_accuracies: Dict[str, float]) -> Dict[str, float]:
    """
    Génère et affiche les graphiques d'évolution de l'accuracy et d'amélioration de l'accuracy pour un dataset donné.
    
    Args:
        dataset_name (str): Nom du dataset.
        methods (list): Liste des méthodes utilisées.
        accuracies (dict): Dictionnaire des accuracies obtenues par méthode pour chaque dataset.
        labeled_ratio (float): Ratio initial de données labellisées.
        batch_ratio (float): Ratio de batch d'ajout de labels à chaque itération.
        n_iterations (int): Nombre total d'itérations.
        full_training_accuracies (dict): Accuracy du modèle entraîné sur tout le training set pour chaque dataset.
        final_accuracies (dict): Dictionnaire des accuracies finales obtenues par chaque méthode après toutes les itérations.
        initial_accuracies (dict): Dictionnaire des accuracies initiales avant le processus d'Active Learning.
    
    Returns:
        dict: Dictionnaire des améliorations d'accuracy pour chaque méthode.
    """
    
    # === Courbe d'évolution de l'accuracy ===
    fig = px.line(title=f"Évolution de l'accuracy sur {dataset_name}")
    
    # Tracer l'évolution de l'accuracy pour chaque méthode
    for method in methods:
        # Ajout de l'accuracy initiale avant la première itération
        x_values = [(labeled_ratio + i * batch_ratio) * 100 for i in range(n_iterations + 1)]
        y_values = [initial_accuracies[dataset_name]] + accuracies[dataset_name][method]
        
        fig.add_scatter(x=x_values, y=y_values, mode='lines+markers', name=method)
    
    # Ajouter la performance du modèle entraîné sur tout le training set
    fig.add_scatter(
        x=[labeled_ratio * 100, (labeled_ratio + (n_iterations+1) * batch_ratio) * 100],
        y=[full_training_accuracies[dataset_name]] * 2,
        mode='lines', name="Full Training Set", line=dict(dash='dash')
    )
    
    # Mise en forme du graphique : axes et légende
    fig.update_layout(
        xaxis_title="% du training set labellisé",
        yaxis_title="Accuracy",
        xaxis=dict(range=[labeled_ratio * 100, (labeled_ratio + n_iterations * batch_ratio) * 100]),
        yaxis=dict(range=[min(min(accuracies[dataset_name][m]) for m in methods) - 0.02, 
                          max(full_training_accuracies[dataset_name], 
                              max(final_accuracies[dataset_name].values())) + 0.02])
    )
    fig.show()
    
    # === Barres d'amélioration de l'accuracy ===
    # Calculer l'amélioration de l'accuracy pour chaque méthode
    accuracy_improvements = {
        method: final_accuracies[dataset_name][method] - initial_accuracies[dataset_name] 
        for method in methods
    }
    
    # Tracer un graphique en barres montrant l'amélioration de l'accuracy
    fig = px.bar(
        x=methods, 
        y=[accuracy_improvements[m] for m in methods],
        labels={"x": "Méthode", "y": "Amélioration d'Accuracy"},
        title=f"Amélioration de l'Accuracy sur {dataset_name}"
    )
    
    # Ajuster les limites de l'axe des y pour un meilleur affichage
    fig.update_layout(
        yaxis=dict(range=[min(accuracy_improvements.values()) - 0.01, 
                          max(accuracy_improvements.values()) + 0.01])
    )
    fig.show()
    
    return accuracy_improvements


In [131]:
def run_active_learning_experiment_datasets(datasets: dict, labeled_ratio: float, test_ratio: float, 
                                            n_iterations: int, batch_ratio: float, methods: list, 
                                            model_class: type, models: list) -> dict:
    """
    Exécute une expérimentation d'Active Learning sur plusieurs datasets avec différentes méthodes de sélection d'incertitude.
    
    Paramètres:
        datasets (dict): Un dictionnaire contenant les datasets sous la forme {nom_dataset: (X, y)},
                         où X est un np.ndarray (features) et y est un np.ndarray (labels).
        labeled_ratio (float): Proportion initiale d'échantillons labellisés dans l'ensemble d'entraînement.
        test_ratio (float): Proportion d'échantillons dédiés à l'ensemble de test.
        n_iterations (int): Nombre d'itérations d'Active Learning.
        batch_ratio (float): Proportion d'échantillons ajoutés à chaque itération (par rapport à la taille totale du training dataset).
        methods (list): Liste des méthodes de sélection d'incertitude à tester.
        model_class (type): Classe du modèle de classification utilisé (doit être compatible avec `fit` et `predict`).
        models (list): Liste des modèles pour les méthodes basées sur un comité (ex: Query By Committee).

    Retourne:
        dict: Un dictionnaire contenant les précisions finales pour chaque dataset et chaque méthode d'Active Learning.
    """
    
    # Initialisation des dictionnaires pour stocker les résultats
    accuracies = {dataset_name: {method: [] for method in methods} for dataset_name in datasets.keys()}
    final_accuracies = {dataset_name: {} for dataset_name in datasets.keys()}
    initial_accuracies = {}
    full_training_accuracies = {}
    
    for dataset_name, (X, y) in datasets.items():
        logging.info(f"\nTraitement du dataset: {dataset_name}")
        
        # Création des ensembles d'entraînement et de test
        X_train, X_pool, y_train, y_pool, X_test, y_test = split_dataset(X, y, labeled_ratio, test_ratio)
        batch_size = int(batch_ratio * (X_train.shape[0]+X_pool.shape[0]))
        
        total_samples = X.shape[0]
        labeled_percentage = (X_train.shape[0] /(X_train.shape[0]+X_pool.shape[0])) * 100
        unlabeled_percentage = (X_pool.shape[0] / (X_train.shape[0]+X_pool.shape[0])) * 100
        test_percentage = (X_test.shape[0] / total_samples) * 100

        # Affichage des tailles des ensembles
        total_samples = X.shape[0]
        logging.info(f"Taille totale du dataset {dataset_name}: {total_samples/total_samples*100:.2f}% ({total_samples}/{total_samples})")
        logging.info(f"Taille de l'ensemble de test: {test_percentage:.2f}% ({X_test.shape[0]}/{total_samples})")
        logging.info(f"Taille de l'ensemble de training: {100-test_percentage:.2f}% ({X_train.shape[0]+X_pool.shape[0]}/{total_samples})")     
        logging.info(f"Taille de l'ensemble labellisé dans le training set: {labeled_percentage:.2f}% ({X_train.shape[0]}/{X_train.shape[0]+X_pool.shape[0]})")
        logging.info(f"Taille de l'ensemble non-labellisé dans le training set: {unlabeled_percentage:.2f}% ({X_pool.shape[0]}/{X_train.shape[0]+X_pool.shape[0]})")
        logging.info(f"Nb d'itérations: {n_iterations}")
        logging.info(f"Nb de données labellisées en plus à chaque itération: {batch_size/(X_train.shape[0]+X_pool.shape[0])*100:.2f}% ({batch_size}/{X_train.shape[0]+X_pool.shape[0]})")
        
        # Évaluation initiale
        model = model_class()
        y_pred_initial = model.fit(X_train, y_train).predict(X_test)
        initial_accuracies[dataset_name] = accuracy_score(y_test, y_pred_initial)
        
        # Modèle entraîné sur l'ensemble complet des données d'entraînement
        model_full = model_class()
        model_full.fit(np.vstack((X_train, X_pool)), np.hstack((y_train, y_pool)))
        full_training_accuracies[dataset_name] = accuracy_score(y_test, model_full.predict(X_test))
        
        # === Boucle principale d'Active Learning ===
        for method in methods:
            X_train_temp, y_train_temp = X_train.copy(), y_train.copy()
            X_pool_temp, y_pool_temp = X_pool.copy(), y_pool.copy()
            
            for i in range(n_iterations):
                # Entraînement et évaluation du modèle
                model, acc = train_and_evaluate(X_train_temp, y_train_temp, X_test, y_test, model_class)
                accuracies[dataset_name][method].append(acc)
                
                # Vérification de l'existence d'échantillons non labellisés
                if len(X_pool_temp) == 0:
                    logging.info(f"\nToutes les données ont été labellisées après {i} itérations.")
                    break
                
                # Ajustement de la taille du batch si nécessaire
                actual_batch_size = min(batch_size, len(X_pool_temp))
                if actual_batch_size < batch_size:
                    logging.info(f"\nBatch réduit à {actual_batch_size} échantillons car la pool est presque vide.")
                
                # Sélection des échantillons les plus incertains
                uncertain_indices = select_uncertain_samples_general(method, model, X_pool_temp, actual_batch_size, models, X_train_temp, y_train_temp, similarity_metric='cosine')
                
                # Mise à jour des ensembles labellisés et non-labellisés
                X_train_temp, y_train_temp, X_pool_temp, y_pool_temp = update_labeled_unlabeled_sets(
                    X_train_temp, y_train_temp, X_pool_temp, y_pool_temp, uncertain_indices
                )
                
                logging.info(f"{method} - Iteration {i+1}: {len(X_train_temp)/(X_train.shape[0]+X_pool.shape[0])*100:.2f}% ({len(X_train_temp)}/{(X_train.shape[0]+X_pool.shape[0])}) samples labeled, Accuracy: {acc:.4f} on {dataset_name}")
            
            # Calcul de la précision finale après Active Learning
            final_accuracies[dataset_name][method] = accuracy_score(y_test, model.predict(X_test))
            logging.info(f"Final Accuracy ({method}) on {dataset_name}: {final_accuracies[dataset_name][method]:.4f}")

        # Génération des graphiques et affichage des résultats
        accuracy_improvements = plot_active_learning_results(dataset_name, methods, accuracies, labeled_ratio, batch_ratio,i+1, full_training_accuracies, final_accuracies, initial_accuracies)
        
        logging.info(f"Initial Accuracy on {dataset_name}: {initial_accuracies[dataset_name]:.4f}")
        logging.info(f"Full Training Set Accuracy on {dataset_name}: {full_training_accuracies[dataset_name]:.4f}")
        for method in methods:
            logging.info(f"Accuracy Improvement ({method}) on {dataset_name}: {accuracy_improvements[method]:.4f}")
    
    return final_accuracies

Traitement MNIST

In [33]:
def read_images(filename: str) -> np.ndarray:
    """
    Lit les images à partir d'un fichier binaire au format MNIST.
    
    Args:
        filename (str): Le nom du fichier contenant les images.
        
    Returns:
        np.ndarray: Un tableau numpy de forme (nb_images, nb_rows, nb_cols) contenant les images lues.
    
    L'assertion `magic_number == 2051` vérifie que le fichier est bien au format attendu pour les images.
    """
    with open(filename, 'rb') as file:
        # Lire les 16 premiers octets pour obtenir les métadonnées
        magic_number, nb_images, nb_rows, nb_cols = struct.unpack('>IIII', file.read(16))
        
        # Vérification du magic_number (doit être 2051 pour les images MNIST)
        assert magic_number == 2051, "Wrong file format"
        
        # Lire les données d'images et les redimensionner en un tableau 3D (nb_images, nb_rows, nb_cols)
        image_data = np.fromfile(file, dtype=np.uint8).reshape(nb_images, nb_rows, nb_cols)
        
    return image_data

In [34]:
def read_targets(filename: str) -> np.ndarray:
    """
    Lit les cibles (labels) à partir d'un fichier binaire au format MNIST.
    
    Args:
        filename (str): Le nom du fichier contenant les cibles.
        
    Returns:
        np.ndarray: Un tableau numpy contenant les labels des images lues.
    
    L'assertion `magic_number == 2049` vérifie que le fichier est bien au format attendu pour les cibles.
    """
    with open(filename, 'rb') as file:
        # Lire les 8 premiers octets pour obtenir les métadonnées
        magic_number, nb_items = struct.unpack('>II', file.read(8))
        
        # Vérification du magic_number (doit être 2049 pour les cibles MNIST)
        assert magic_number == 2049, "Wrong file format"
        
        # Lire les labels
        targets = np.fromfile(file, dtype=np.uint8)
        
    return targets

In [35]:
path_images_file = '../t10k-images.idx3-ubyte'
path_to_targets = '../t10k-labels.idx1-ubyte'

X_MNIST= read_images(path_images_file).reshape(-1, 28*28)
y_MNIST = read_targets(path_to_targets)

Traitement PRS

In [31]:
color_values = {
    "Yellow": 2,
    "Brown": 2,
    "Red": 2,
    "Black": 2,
    "Grey": 2,
    "Pink": 2,
    "No flag": 0,
    "Blue": 1,
    "Bluish": 1,
    "Purple": 1,
    "Green": 0,
    "Salmon": 1,
}

In [32]:
df1=pd.read_csv("all_reviews_features 2.csv")
df2=pd.read_csv("reviews_since_february_with_features 1.csv")
df3=pd.read_csv("reviews_test_nb_bets 2.csv")
df=pd.concat([df1,df2,df3])

# Charger les datasets
df1 = pd.read_csv("all_reviews_features 2.csv")
df2 = pd.read_csv("reviews_since_february_with_features 1.csv")
df3 = pd.read_csv("reviews_test_nb_bets 2.csv")

# Combinaison des datasets
df = pd.concat([df1, df2, df3])

# Définir les caractéristiques
FEATURES = ['SINGLE_PROPORTION_30D', 'MARKET_MARGIN_30D', 'TURNOVER_PER_BET_30D', 'DEPOSIT_MAX_30D', 
            'SHIELD_REJECTION_30D', 'LIVE_PROPORTION_30D', 'GGR_PER_BET_30D', 'MARGIN_30D', 
            'MARGIN_PER_BET_30D', 'CLOSING_LINE_VALUE_30D', 'MAX_STAKE_RATIO_30D', 'MARKET_TYPE_SCORE_30D', 
            'BET_SCORE_30D', 'TIME_BEFORE_EVENT_30D', 'LATE_BET_ACTION_COUNT_30D', 
            'LATE_BET_STOLEN_AMOUNT_EURO_30D', 'SINGLE_PROPORTION_10D', 'MARKET_MARGIN_10D', 
            'TURNOVER_PER_BET_10D', 'DEPOSIT_MAX_10D', 'SHIELD_REJECTION_10D', 'LIVE_PROPORTION_10D', 
            'GGR_PER_BET_10D', 'MARGIN_10D', 'MARGIN_PER_BET_10D', 'CLOSING_LINE_VALUE_10D', 
            'MAX_STAKE_RATIO_10D', 'MARKET_TYPE_SCORE_10D', 'BET_SCORE_10D', 'TIME_BEFORE_EVENT_10D', 
            'LATE_BET_ACTION_COUNT_10D', 'LATE_BET_STOLEN_AMOUNT_EURO_10D', 'SELECTION_DIVERSITY_LAST_0_DAYS', 
            'SHARP_SHARE_LAST_0_DAYS', 'YELLOW_BREAKOUT_LAST_0_DAYS', 'RISKY_RING_RATIO_LAST_0_DAYS', 
            'SINGLE_PROPORTION_365D', 'MARKET_MARGIN_365D', 'TURNOVER_PER_BET_365D', 'DEPOSIT_MAX_365D', 
            'SHIELD_REJECTION_365D', 'LIVE_PROPORTION_365D', 'GGR_PER_BET_365D', 'MARGIN_PER_BET_365D', 
            'MARGIN_365D', 'CLOSING_LINE_VALUE_365D', 'MARKET_TYPE_SCORE_365D', 'BET_SCORE_365D', 
            'SHARP_SHARE_365D', 'TIME_BEFORE_EVENT_365D', 'MAX_STAKE_RATIO_365D', 'SINGLE_PROPORTION_180D', 
            'MARKET_MARGIN_180D', 'TURNOVER_PER_BET_180D', 'DEPOSIT_MAX_180D', 'SHIELD_REJECTION_180D', 
            'LIVE_PROPORTION_180D', 'GGR_PER_BET_180D', 'MARGIN_PER_BET_180D', 'MARGIN_180D', 
            'CLOSING_LINE_VALUE_180D', 'MARKET_TYPE_SCORE_180D', 'BET_SCORE_180D', 'SHARP_SHARE_180D', 
            'TIME_BEFORE_EVENT_180D', 'MAX_STAKE_RATIO_180D', 'SINGLE_PROPORTION_90D', 'MARKET_MARGIN_90D', 
            'TURNOVER_PER_BET_90D', 'DEPOSIT_MAX_90D', 'SHIELD_REJECTION_90D', 'LIVE_PROPORTION_90D', 
            'GGR_PER_BET_90D', 'MARGIN_PER_BET_90D', 'MARGIN_90D', 'CLOSING_LINE_VALUE_90D', 
            'MARKET_TYPE_SCORE_90D', 'BET_SCORE_90D', 'SHARP_SHARE_90D', 'TIME_BEFORE_EVENT_90D', 
            'MAX_STAKE_RATIO_90D', 'LATE_BET_ACTION_COUNT_90D', 'LATE_BET_STOLEN_AMOUNT_EURO_90D', 
            'SHARP_SHARE_30D', 'SHARP_SHARE_10D', 'has_previous', 'NB_BET_LAST_10_DAYS', 
            'NB_BET_LAST_30_DAYS', 'NB_BET_LAST_90_DAYS', 'NB_BET_LAST_180_DAYS', 'NB_BET_LAST_365_DAYS', 
            'TURNOVER_LAST_10_DAYS', 'TURNOVER_LAST_30_DAYS', 'TURNOVER_LAST_90_DAYS', 
            'TURNOVER_LAST_180_DAYS', 'TURNOVER_LAST_365_DAYS', 'DEPOSIT_LAST_10_DAYS', 
            'DEPOSIT_LAST_30_DAYS', 'DEPOSIT_LAST_90_DAYS', 'DEPOSIT_LAST_180_DAYS', 'DEPOSIT_LAST_365_DAYS', 
            'GGR_LAST_10_DAYS', 'GGR_LAST_30_DAYS', 'GGR_LAST_90_DAYS', 'GGR_LAST_180_DAYS', 
            'GGR_LAST_365_DAYS', 'ORIGINAL_FLAG']

# Mapper les colonnes 'CURRENT_FLAG' et 'ORIGINAL_FLAG' avec color_values
color_values = {
    "Yellow": 2,
    "Brown": 2,
    "Red": 2,
    "Black": 2,
    "Grey": 2,
    "Pink": 2,
    "No flag": 0,
    "Blue": 1,
    "Bluish": 1,
    "Purple": 1,
    "Green": 0,
    "Salmon": 1,
}
df['label'] = df['CURRENT_FLAG'].map(color_values).fillna(0)  # Remplacement et gestion des valeurs manquantes
df['ORIGINAL_FLAG'] = df['ORIGINAL_FLAG'].map(color_values).fillna(0)
df['label'] = df['label'].replace(1, 0)  # Correction spécifique si nécessaire

# Extraire les features et labels
X_PRS = df[FEATURES]
y_PRS = df["label"]

# Conversion en numpy arrays
X_PRS = X_PRS.to_numpy()
y_PRS = y_PRS.to_numpy()

# Affichage des dimensions des données
print("X_PRS shape:", X_PRS.shape)
print("y_PRS shape:", y_PRS.shape)
print("Distribution des labels :")
print(pd.Series(y_PRS).value_counts())


X_PRS shape: (22250, 107)
y_PRS shape: (22250,)
Distribution des labels :
0.0    18373
2.0     3877
Name: count, dtype: int64


Dataset foot : https://www.kaggle.com/datasets/vivovinco/20222023-football-player-stats/data

In [129]:
df=pd.read_csv("2022-2023 Football Player Stats.csv",delimiter=";")
df=df.drop(['Rk','Player','Nation','Squad','Comp'],axis=1)
# Créer une instance de LabelEncoder
label_encoder = LabelEncoder()

# Appliquer fit_transform à la colonne 'Pos' et assigner les valeurs encodées à la nouvelle colonne 'Pos'
df['Pos'] = label_encoder.fit_transform(df['Pos'])

X_foot=df.drop("Pos",axis=1).to_numpy()
y_foot=df["Pos"].to_numpy()


# Appliquer SMOTE
smote = SMOTE(sampling_strategy={0: 959, 1: 950, 2: 939, 3: 910, 4: 909, 5: 910, 6: 909, 7: 910, 8: 889, 9: 895}, random_state=42)
X_foot, y_foot = smote.fit_resample(X_foot, y_foot)

print(np.unique(y_foot,return_counts=True),
X_foot.shape)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([959, 950, 939, 910, 909, 910, 909, 910, 889, 895])) (9180, 118)


3% initialement labellisé. Ajout de 1% à chaque itération jusqu'à labelliser tout le training set

In [36]:

labeled_ratio=0.03 # Nombre d'échantillons labellisés initialement
test_ratio = 0.2  # Proportion du dataset réservée au test
n_iterations = 100 # Nombre d'itérations d'Active Learning
batch_ratio =0.01 # Taille du batch d'échantillons ajoutés à chaque itération
#methods=["random", "least_confident", "margin", "entropy","hybrid","qbc-variance","qbc-entropy","qbc-KL"]
methods=["random", "least_confident", "margin", "entropy"]
model_class=lambda : RandomForestClassifier()
# Créer un comité avec 3 modèles différents
# Liste de modèles pour le comité
models = [
    clone(RandomForestClassifier()),  # Random Forest
    clone(LogisticRegression(max_iter=1000)), # Régression Logistique
    clone(SVC(probability=True)),      # Régression Ridge pour classificati)           # Analyse discriminante quadratique
]
datasets={"MNIST":(X_MNIST,y_MNIST),"PRS":(X_PRS,y_PRS),"Amazon":(X_Amazon,y_Amazon)}
#datasets={"MNIST":(X_MNIST,y_MNIST),"PRS":(X_PRS,y_PRS)}
# Exécution de l'expérience
run_active_learning_experiment_datasets(datasets, labeled_ratio, test_ratio, n_iterations, batch_ratio, methods, model_class, models)


Traitement du dataset: MNIST
Taille totale du dataset MNIST: 100.00% (10000/10000)
Taille de l'ensemble de test: 20.00% (2000/10000)
Taille de l'ensemble de training: 80.00% (8000/10000)
Taille de l'ensemble labellisé dans le training set: 3.00% (300/8000)
Taille de l'ensemble non-labellisé dans le training set: 77.00% (7700/8000)
Nb d'itérations: 100
Nb de données labellisées en plus à chaque itération: 1.00% (80/8000)
random - Iteration 1: 4.75% (380/8000) samples labeled, Accuracy: 0.8040 on MNIST
random - Iteration 2: 5.75% (460/8000) samples labeled, Accuracy: 0.8435 on MNIST
random - Iteration 3: 6.75% (540/8000) samples labeled, Accuracy: 0.8705 on MNIST
random - Iteration 4: 7.75% (620/8000) samples labeled, Accuracy: 0.8700 on MNIST
random - Iteration 5: 8.75% (700/8000) samples labeled, Accuracy: 0.8690 on MNIST
random - Iteration 6: 9.75% (780/8000) samples labeled, Accuracy: 0.8730 on MNIST
random - Iteration 7: 10.75% (860/8000) samples labeled, Accuracy: 0.8910 on MNIST


Initial Accuracy on MNIST: 0.8230
Full Training Set Accuracy on MNIST: 0.9490
Accuracy Improvement (random) on MNIST: 0.1250
Accuracy Improvement (least_confident) on MNIST: 0.1315
Accuracy Improvement (margin) on MNIST: 0.1235
Accuracy Improvement (entropy) on MNIST: 0.1240

Traitement du dataset: PRS
Taille totale du dataset PRS: 100.00% (22250/22250)
Taille de l'ensemble de test: 20.00% (4450/22250)
Taille de l'ensemble de training: 80.00% (17800/22250)
Taille de l'ensemble labellisé dans le training set: 3.00% (667/17800)
Taille de l'ensemble non-labellisé dans le training set: 77.00% (17133/17800)
Nb d'itérations: 100
Nb de données labellisées en plus à chaque itération: 1.00% (178/17800)
random - Iteration 1: 4.75% (845/17800) samples labeled, Accuracy: 0.8746 on PRS
random - Iteration 2: 5.75% (1023/17800) samples labeled, Accuracy: 0.8755 on PRS
random - Iteration 3: 6.75% (1201/17800) samples labeled, Accuracy: 0.8780 on PRS
random - Iteration 4: 7.75% (1379/17800) samples lab

Initial Accuracy on PRS: 0.8742
Full Training Set Accuracy on PRS: 0.8993
Accuracy Improvement (random) on PRS: 0.0211
Accuracy Improvement (least_confident) on PRS: 0.0234
Accuracy Improvement (margin) on PRS: 0.0236
Accuracy Improvement (entropy) on PRS: 0.0229

Traitement du dataset: Amazon
Taille totale du dataset Amazon: 100.00% (10000/10000)
Taille de l'ensemble de test: 20.00% (2000/10000)
Taille de l'ensemble de training: 80.00% (8000/10000)
Taille de l'ensemble labellisé dans le training set: 3.00% (300/8000)
Taille de l'ensemble non-labellisé dans le training set: 77.00% (7700/8000)
Nb d'itérations: 100
Nb de données labellisées en plus à chaque itération: 1.00% (80/8000)
random - Iteration 1: 4.75% (380/8000) samples labeled, Accuracy: 0.3305 on Amazon
random - Iteration 2: 5.75% (460/8000) samples labeled, Accuracy: 0.3340 on Amazon
random - Iteration 3: 6.75% (540/8000) samples labeled, Accuracy: 0.3445 on Amazon
random - Iteration 4: 7.75% (620/8000) samples labeled, Accu

Initial Accuracy on Amazon: 0.3135
Full Training Set Accuracy on Amazon: 0.5060
Accuracy Improvement (random) on Amazon: 0.1930
Accuracy Improvement (least_confident) on Amazon: 0.1915
Accuracy Improvement (margin) on Amazon: 0.1960
Accuracy Improvement (entropy) on Amazon: 0.1795


{'MNIST': {'random': 0.948,
  'least_confident': 0.9545,
  'margin': 0.9465,
  'entropy': 0.947},
 'PRS': {'random': 0.8952808988764045,
  'least_confident': 0.8975280898876404,
  'margin': 0.8977528089887641,
  'entropy': 0.8970786516853932},
 'Amazon': {'random': 0.5065,
  'least_confident': 0.505,
  'margin': 0.5095,
  'entropy': 0.493}}

In [132]:

labeled_ratio=0.03 # Nombre d'échantillons labellisés initialement
test_ratio = 0.2  # Proportion du dataset réservée au test
n_iterations = 100# Nombre d'itérations d'Active Learning
batch_ratio =0.01 # Taille du batch d'échantillons ajoutés à chaque itération
#methods=["random", "least_confident", "margin", "entropy","hybrid","qbc-variance","qbc-entropy","qbc-KL"]
methods=["random", "least_confident", "margin", "entropy"]
model_class=lambda : RandomForestClassifier()
# Créer un comité avec 3 modèles différents
# Liste de modèles pour le comité
models = [
    clone(RandomForestClassifier()),  # Random Forest
    clone(LogisticRegression(max_iter=1000)), # Régression Logistique
    clone(SVC(probability=True)),      # Régression Ridge pour classificati)           # Analyse discriminante quadratique
]
datasets={"Foot":(X_foot,y_foot)}
#datasets={"MNIST":(X_MNIST,y_MNIST),"PRS":(X_PRS,y_PRS)}
# Exécution de l'expérience
run_active_learning_experiment_datasets(datasets, labeled_ratio, test_ratio, n_iterations, batch_ratio, methods, model_class, models)


Traitement du dataset: Foot
Taille totale du dataset Foot: 100.00% (9180/9180)
Taille de l'ensemble de test: 20.00% (1836/9180)
Taille de l'ensemble de training: 80.00% (7344/9180)
Taille de l'ensemble labellisé dans le training set: 3.74% (275/7344)
Taille de l'ensemble non-labellisé dans le training set: 96.26% (7069/7344)
Nb d'itérations: 100
Nb de données labellisées en plus à chaque itération: 0.99% (73/7344)
random - Iteration 1: 4.74% (348/7344) samples labeled, Accuracy: 0.5931 on Foot
random - Iteration 2: 5.73% (421/7344) samples labeled, Accuracy: 0.6356 on Foot
random - Iteration 3: 6.73% (494/7344) samples labeled, Accuracy: 0.6536 on Foot
random - Iteration 4: 7.72% (567/7344) samples labeled, Accuracy: 0.6819 on Foot
random - Iteration 5: 8.71% (640/7344) samples labeled, Accuracy: 0.7015 on Foot
random - Iteration 6: 9.71% (713/7344) samples labeled, Accuracy: 0.7315 on Foot
random - Iteration 7: 10.70% (786/7344) samples labeled, Accuracy: 0.7484 on Foot
random - Iter

Initial Accuracy on Foot: 0.6051
Full Training Set Accuracy on Foot: 0.9395
Accuracy Improvement (random) on Foot: 0.3339
Accuracy Improvement (least_confident) on Foot: 0.3333
Accuracy Improvement (margin) on Foot: 0.3295
Accuracy Improvement (entropy) on Foot: 0.3333


{'Foot': {'random': 0.9389978213507625,
  'least_confident': 0.9384531590413944,
  'margin': 0.934640522875817,
  'entropy': 0.9384531590413944}}

évolution de l'accuracy en donnant jusqu'à 30% du training set avec différents batch_ratio. 1% initialement labellisé

In [133]:
# Paramètres fixes
labeled_ratio = 0.01  # 1% de données labellisées initialement
test_ratio = 0.2  # Proportion du dataset réservée au test
methods = ["random", "least_confident", "margin", "entropy"]
model_class = lambda: RandomForestClassifier()
datasets={"MNIST":(X_MNIST,y_MNIST)}

# Liste des batch ratios à tester
batch_ratios = [0.01,0.02,0.05,0.075,0.1,0.5]

# Stockage des résultats
accuracy_results = {}

for batch_ratio in batch_ratios:
    # Calculer le nombre d'itérations nécessaires pour atteindre 30%
    n_iterations = int(np.ceil((0.30 - labeled_ratio) / batch_ratio))
    
    print(f"Testing batch_ratio = {batch_ratio} with n_iterations = {n_iterations}")
    
    # Exécuter l'expérience
    accuracy_results[batch_ratio] = run_active_learning_experiment_datasets(
        datasets, labeled_ratio, test_ratio, n_iterations, batch_ratio, methods, model_class, models
    )





Traitement du dataset: MNIST
Taille totale du dataset MNIST: 100.00% (10000/10000)
Taille de l'ensemble de test: 20.00% (2000/10000)
Taille de l'ensemble de training: 80.00% (8000/10000)
Taille de l'ensemble labellisé dans le training set: 1.25% (100/8000)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (7900/8000)
Nb d'itérations: 29
Nb de données labellisées en plus à chaque itération: 1.00% (80/8000)


Testing batch_ratio = 0.01 with n_iterations = 29


random - Iteration 1: 2.25% (180/8000) samples labeled, Accuracy: 0.6910 on MNIST
random - Iteration 2: 3.25% (260/8000) samples labeled, Accuracy: 0.8015 on MNIST
random - Iteration 3: 4.25% (340/8000) samples labeled, Accuracy: 0.8340 on MNIST
random - Iteration 4: 5.25% (420/8000) samples labeled, Accuracy: 0.8465 on MNIST
random - Iteration 5: 6.25% (500/8000) samples labeled, Accuracy: 0.8635 on MNIST
random - Iteration 6: 7.25% (580/8000) samples labeled, Accuracy: 0.8690 on MNIST
random - Iteration 7: 8.25% (660/8000) samples labeled, Accuracy: 0.8840 on MNIST
random - Iteration 8: 9.25% (740/8000) samples labeled, Accuracy: 0.8850 on MNIST
random - Iteration 9: 10.25% (820/8000) samples labeled, Accuracy: 0.8890 on MNIST
random - Iteration 10: 11.25% (900/8000) samples labeled, Accuracy: 0.8865 on MNIST
random - Iteration 11: 12.25% (980/8000) samples labeled, Accuracy: 0.8950 on MNIST
random - Iteration 12: 13.25% (1060/8000) samples labeled, Accuracy: 0.9005 on MNIST
random -

Initial Accuracy on MNIST: 0.7070
Full Training Set Accuracy on MNIST: 0.9505
Accuracy Improvement (random) on MNIST: 0.2175
Accuracy Improvement (least_confident) on MNIST: 0.2510
Accuracy Improvement (margin) on MNIST: 0.2485
Accuracy Improvement (entropy) on MNIST: 0.2415

Traitement du dataset: MNIST
Taille totale du dataset MNIST: 100.00% (10000/10000)
Taille de l'ensemble de test: 20.00% (2000/10000)
Taille de l'ensemble de training: 80.00% (8000/10000)
Taille de l'ensemble labellisé dans le training set: 1.25% (100/8000)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (7900/8000)
Nb d'itérations: 15
Nb de données labellisées en plus à chaque itération: 2.00% (160/8000)


Testing batch_ratio = 0.02 with n_iterations = 15


...

random - Iteration 1: 11.25% (900/8000) samples labeled, Accuracy: 0.6940 on MNIST
random - Iteration 2: 21.25% (1700/8000) samples labeled, Accuracy: 0.8960 on MNIST
random - Iteration 3: 31.25% (2500/8000) samples labeled, Accuracy: 0.9140 on MNIST
Final Accuracy (random) on MNIST: 0.9140
least_confident - Iteration 1: 11.25% (900/8000) samples labeled, Accuracy: 0.6905 on MNIST
least_confident - Iteration 2: 21.25% (1700/8000) samples labeled, Accuracy: 0.8685 on MNIST
least_confident - Iteration 3: 31.25% (2500/8000) samples labeled, Accuracy: 0.9285 on MNIST
Final Accuracy (least_confident) on MNIST: 0.9285
margin - Iteration 1: 11.25% (900/8000) samples labeled, Accuracy: 0.6955 on MNIST
margin - Iteration 2: 21.25% (1700/8000) samples labeled, Accuracy: 0.8725 on MNIST
margin - Iteration 3: 31.25% (2500/8000) samples labeled, Accuracy: 0.9325 on MNIST
Final Accuracy (margin) on MNIST: 0.9325
entropy - Iteration 1: 11.25% (900/8000) samples labeled, Accuracy: 0.6975 on MNIST
entr

Initial Accuracy on MNIST: 0.6805
Full Training Set Accuracy on MNIST: 0.9490
Accuracy Improvement (random) on MNIST: 0.2335
Accuracy Improvement (least_confident) on MNIST: 0.2480
Accuracy Improvement (margin) on MNIST: 0.2520
Accuracy Improvement (entropy) on MNIST: 0.2430

Traitement du dataset: MNIST
Taille totale du dataset MNIST: 100.00% (10000/10000)
Taille de l'ensemble de test: 20.00% (2000/10000)
Taille de l'ensemble de training: 80.00% (8000/10000)
Taille de l'ensemble labellisé dans le training set: 1.25% (100/8000)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (7900/8000)
Nb d'itérations: 1
Nb de données labellisées en plus à chaque itération: 50.00% (4000/8000)


Testing batch_ratio = 0.5 with n_iterations = 1


random - Iteration 1: 51.25% (4100/8000) samples labeled, Accuracy: 0.6960 on MNIST
Final Accuracy (random) on MNIST: 0.6960
least_confident - Iteration 1: 51.25% (4100/8000) samples labeled, Accuracy: 0.6965 on MNIST
Final Accuracy (least_confident) on MNIST: 0.6965
margin - Iteration 1: 51.25% (4100/8000) samples labeled, Accuracy: 0.6890 on MNIST
Final Accuracy (margin) on MNIST: 0.6890
entropy - Iteration 1: 51.25% (4100/8000) samples labeled, Accuracy: 0.7035 on MNIST
Final Accuracy (entropy) on MNIST: 0.7035


Initial Accuracy on MNIST: 0.6765
Full Training Set Accuracy on MNIST: 0.9470
Accuracy Improvement (random) on MNIST: 0.0195
Accuracy Improvement (least_confident) on MNIST: 0.0200
Accuracy Improvement (margin) on MNIST: 0.0125
Accuracy Improvement (entropy) on MNIST: 0.0270


In [134]:
# Paramètres fixes
labeled_ratio = 0.01  # 1% de données labellisées initialement
test_ratio = 0.2  # Proportion du dataset réservée au test
methods = ["random", "least_confident", "margin", "entropy"]
model_class = lambda: RandomForestClassifier()
datasets={"PRS":(X_PRS,y_PRS)}

# Liste des batch ratios à tester
batch_ratios = [0.01,0.02,0.05,0.075,0.1,0.5]

# Stockage des résultats
accuracy_results = {}

for batch_ratio in batch_ratios:
    # Calculer le nombre d'itérations nécessaires pour atteindre 30%
    n_iterations = int(np.ceil((0.30 - labeled_ratio) / batch_ratio))
    
    print(f"Testing batch_ratio = {batch_ratio} with n_iterations = {n_iterations}")
    
    # Exécuter l'expérience
    accuracy_results[batch_ratio] = run_active_learning_experiment_datasets(
        datasets, labeled_ratio, test_ratio, n_iterations, batch_ratio, methods, model_class, models
    )





Traitement du dataset: PRS
Taille totale du dataset PRS: 100.00% (22250/22250)
Taille de l'ensemble de test: 20.00% (4450/22250)
Taille de l'ensemble de training: 80.00% (17800/22250)
Taille de l'ensemble labellisé dans le training set: 1.25% (222/17800)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (17578/17800)
Nb d'itérations: 29
Nb de données labellisées en plus à chaque itération: 1.00% (178/17800)


Testing batch_ratio = 0.01 with n_iterations = 29


random - Iteration 1: 2.25% (400/17800) samples labeled, Accuracy: 0.8533 on PRS
random - Iteration 2: 3.25% (578/17800) samples labeled, Accuracy: 0.8670 on PRS
random - Iteration 3: 4.25% (756/17800) samples labeled, Accuracy: 0.8663 on PRS
random - Iteration 4: 5.25% (934/17800) samples labeled, Accuracy: 0.8721 on PRS
random - Iteration 5: 6.25% (1112/17800) samples labeled, Accuracy: 0.8735 on PRS
random - Iteration 6: 7.25% (1290/17800) samples labeled, Accuracy: 0.8766 on PRS
random - Iteration 7: 8.25% (1468/17800) samples labeled, Accuracy: 0.8807 on PRS
random - Iteration 8: 9.25% (1646/17800) samples labeled, Accuracy: 0.8807 on PRS
random - Iteration 9: 10.25% (1824/17800) samples labeled, Accuracy: 0.8825 on PRS
random - Iteration 10: 11.25% (2002/17800) samples labeled, Accuracy: 0.8811 on PRS
random - Iteration 11: 12.25% (2180/17800) samples labeled, Accuracy: 0.8840 on PRS
random - Iteration 12: 13.25% (2358/17800) samples labeled, Accuracy: 0.8854 on PRS
random - Iter

Initial Accuracy on PRS: 0.8557
Full Training Set Accuracy on PRS: 0.8993
Accuracy Improvement (random) on PRS: 0.0380
Accuracy Improvement (least_confident) on PRS: 0.0416
Accuracy Improvement (margin) on PRS: 0.0407
Accuracy Improvement (entropy) on PRS: 0.0431

Traitement du dataset: PRS
Taille totale du dataset PRS: 100.00% (22250/22250)
Taille de l'ensemble de test: 20.00% (4450/22250)
Taille de l'ensemble de training: 80.00% (17800/22250)
Taille de l'ensemble labellisé dans le training set: 1.25% (222/17800)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (17578/17800)
Nb d'itérations: 15
Nb de données labellisées en plus à chaque itération: 2.00% (356/17800)


Testing batch_ratio = 0.02 with n_iterations = 15


...

random - Iteration 1: 11.25% (2002/17800) samples labeled, Accuracy: 0.8555 on PRS
random - Iteration 2: 21.25% (3782/17800) samples labeled, Accuracy: 0.8843 on PRS
random - Iteration 3: 31.25% (5562/17800) samples labeled, Accuracy: 0.8919 on PRS
Final Accuracy (random) on PRS: 0.8919
least_confident - Iteration 1: 11.25% (2002/17800) samples labeled, Accuracy: 0.8591 on PRS
least_confident - Iteration 2: 21.25% (3782/17800) samples labeled, Accuracy: 0.8876 on PRS
least_confident - Iteration 3: 31.25% (5562/17800) samples labeled, Accuracy: 0.8890 on PRS
Final Accuracy (least_confident) on PRS: 0.8890
margin - Iteration 1: 11.25% (2002/17800) samples labeled, Accuracy: 0.8566 on PRS
margin - Iteration 2: 21.25% (3782/17800) samples labeled, Accuracy: 0.8883 on PRS
margin - Iteration 3: 31.25% (5562/17800) samples labeled, Accuracy: 0.8928 on PRS
Final Accuracy (margin) on PRS: 0.8928
entropy - Iteration 1: 11.25% (2002/17800) samples labeled, Accuracy: 0.8548 on PRS
entropy - Iterat

Initial Accuracy on PRS: 0.8524
Full Training Set Accuracy on PRS: 0.8973
Accuracy Improvement (random) on PRS: 0.0396
Accuracy Improvement (least_confident) on PRS: 0.0366
Accuracy Improvement (margin) on PRS: 0.0404
Accuracy Improvement (entropy) on PRS: 0.0312

Traitement du dataset: PRS
Taille totale du dataset PRS: 100.00% (22250/22250)
Taille de l'ensemble de test: 20.00% (4450/22250)
Taille de l'ensemble de training: 80.00% (17800/22250)
Taille de l'ensemble labellisé dans le training set: 1.25% (222/17800)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (17578/17800)
Nb d'itérations: 1
Nb de données labellisées en plus à chaque itération: 50.00% (8900/17800)


Testing batch_ratio = 0.5 with n_iterations = 1


random - Iteration 1: 51.25% (9122/17800) samples labeled, Accuracy: 0.8546 on PRS
Final Accuracy (random) on PRS: 0.8546
least_confident - Iteration 1: 51.25% (9122/17800) samples labeled, Accuracy: 0.8542 on PRS
Final Accuracy (least_confident) on PRS: 0.8542
margin - Iteration 1: 51.25% (9122/17800) samples labeled, Accuracy: 0.8521 on PRS
Final Accuracy (margin) on PRS: 0.8521
entropy - Iteration 1: 51.25% (9122/17800) samples labeled, Accuracy: 0.8548 on PRS
Final Accuracy (entropy) on PRS: 0.8548


Initial Accuracy on PRS: 0.8528
Full Training Set Accuracy on PRS: 0.8960
Accuracy Improvement (random) on PRS: 0.0018
Accuracy Improvement (least_confident) on PRS: 0.0013
Accuracy Improvement (margin) on PRS: -0.0007
Accuracy Improvement (entropy) on PRS: 0.0020


In [135]:
# Paramètres fixes
labeled_ratio = 0.01  # 1% de données labellisées initialement
test_ratio = 0.2  # Proportion du dataset réservée au test
methods = ["random", "least_confident", "margin", "entropy"]
model_class = lambda: RandomForestClassifier()
datasets={"Foot":(X_foot,y_foot)}

# Liste des batch ratios à tester
batch_ratios = [0.01,0.02,0.05,0.075,0.1,0.5]

# Stockage des résultats
accuracy_results = {}

for batch_ratio in batch_ratios:
    # Calculer le nombre d'itérations nécessaires pour atteindre 30%
    n_iterations = int(np.ceil((0.30 - labeled_ratio) / batch_ratio))
    
    print(f"Testing batch_ratio = {batch_ratio} with n_iterations = {n_iterations}")
    
    # Exécuter l'expérience
    accuracy_results[batch_ratio] = run_active_learning_experiment_datasets(
        datasets, labeled_ratio, test_ratio, n_iterations, batch_ratio, methods, model_class, models
    )





Traitement du dataset: Foot
Taille totale du dataset Foot: 100.00% (9180/9180)
Taille de l'ensemble de test: 20.00% (1836/9180)
Taille de l'ensemble de training: 80.00% (7344/9180)
Taille de l'ensemble labellisé dans le training set: 1.24% (91/7344)
Taille de l'ensemble non-labellisé dans le training set: 98.76% (7253/7344)
Nb d'itérations: 29
Nb de données labellisées en plus à chaque itération: 0.99% (73/7344)


Testing batch_ratio = 0.01 with n_iterations = 29


random - Iteration 1: 2.23% (164/7344) samples labeled, Accuracy: 0.4417 on Foot
random - Iteration 2: 3.23% (237/7344) samples labeled, Accuracy: 0.5125 on Foot
random - Iteration 3: 4.22% (310/7344) samples labeled, Accuracy: 0.5888 on Foot
random - Iteration 4: 5.22% (383/7344) samples labeled, Accuracy: 0.6057 on Foot
random - Iteration 5: 6.21% (456/7344) samples labeled, Accuracy: 0.6264 on Foot
random - Iteration 6: 7.20% (529/7344) samples labeled, Accuracy: 0.6514 on Foot
random - Iteration 7: 8.20% (602/7344) samples labeled, Accuracy: 0.6716 on Foot
random - Iteration 8: 9.19% (675/7344) samples labeled, Accuracy: 0.6830 on Foot
random - Iteration 9: 10.19% (748/7344) samples labeled, Accuracy: 0.7048 on Foot
random - Iteration 10: 11.18% (821/7344) samples labeled, Accuracy: 0.7135 on Foot
random - Iteration 11: 12.17% (894/7344) samples labeled, Accuracy: 0.7309 on Foot
random - Iteration 12: 13.17% (967/7344) samples labeled, Accuracy: 0.7429 on Foot
random - Iteration 13

Initial Accuracy on Foot: 0.4488
Full Training Set Accuracy on Foot: 0.9330
Accuracy Improvement (random) on Foot: 0.4020
Accuracy Improvement (least_confident) on Foot: 0.4521
Accuracy Improvement (margin) on Foot: 0.4777
Accuracy Improvement (entropy) on Foot: 0.3981

Traitement du dataset: Foot
Taille totale du dataset Foot: 100.00% (9180/9180)
Taille de l'ensemble de test: 20.00% (1836/9180)
Taille de l'ensemble de training: 80.00% (7344/9180)
Taille de l'ensemble labellisé dans le training set: 1.24% (91/7344)
Taille de l'ensemble non-labellisé dans le training set: 98.76% (7253/7344)
Nb d'itérations: 15
Nb de données labellisées en plus à chaque itération: 1.99% (146/7344)


Testing batch_ratio = 0.02 with n_iterations = 15


...

random - Iteration 1: 11.23% (825/7344) samples labeled, Accuracy: 0.4199 on Foot
random - Iteration 2: 21.23% (1559/7344) samples labeled, Accuracy: 0.7456 on Foot
random - Iteration 3: 31.22% (2293/7344) samples labeled, Accuracy: 0.8246 on Foot
Final Accuracy (random) on Foot: 0.8246
least_confident - Iteration 1: 11.23% (825/7344) samples labeled, Accuracy: 0.4232 on Foot
least_confident - Iteration 2: 21.23% (1559/7344) samples labeled, Accuracy: 0.6269 on Foot
least_confident - Iteration 3: 31.22% (2293/7344) samples labeled, Accuracy: 0.7582 on Foot
Final Accuracy (least_confident) on Foot: 0.7582
margin - Iteration 1: 11.23% (825/7344) samples labeled, Accuracy: 0.4325 on Foot
margin - Iteration 2: 21.23% (1559/7344) samples labeled, Accuracy: 0.7151 on Foot
margin - Iteration 3: 31.22% (2293/7344) samples labeled, Accuracy: 0.8371 on Foot
Final Accuracy (margin) on Foot: 0.8371
entropy - Iteration 1: 11.23% (825/7344) samples labeled, Accuracy: 0.4188 on Foot
entropy - Iterati

Initial Accuracy on Foot: 0.4063
Full Training Set Accuracy on Foot: 0.9401
Accuracy Improvement (random) on Foot: 0.4183
Accuracy Improvement (least_confident) on Foot: 0.3519
Accuracy Improvement (margin) on Foot: 0.4308
Accuracy Improvement (entropy) on Foot: 0.3181

Traitement du dataset: Foot
Taille totale du dataset Foot: 100.00% (9180/9180)
Taille de l'ensemble de test: 20.00% (1836/9180)
Taille de l'ensemble de training: 80.00% (7344/9180)
Taille de l'ensemble labellisé dans le training set: 1.24% (91/7344)
Taille de l'ensemble non-labellisé dans le training set: 98.76% (7253/7344)
Nb d'itérations: 1
Nb de données labellisées en plus à chaque itération: 50.00% (3672/7344)


Testing batch_ratio = 0.5 with n_iterations = 1


random - Iteration 1: 51.24% (3763/7344) samples labeled, Accuracy: 0.4150 on Foot
Final Accuracy (random) on Foot: 0.4150
least_confident - Iteration 1: 51.24% (3763/7344) samples labeled, Accuracy: 0.4308 on Foot
Final Accuracy (least_confident) on Foot: 0.4308
margin - Iteration 1: 51.24% (3763/7344) samples labeled, Accuracy: 0.4368 on Foot
Final Accuracy (margin) on Foot: 0.4368
entropy - Iteration 1: 51.24% (3763/7344) samples labeled, Accuracy: 0.4417 on Foot
Final Accuracy (entropy) on Foot: 0.4417


Initial Accuracy on Foot: 0.4216
Full Training Set Accuracy on Foot: 0.9395
Accuracy Improvement (random) on Foot: -0.0065
Accuracy Improvement (least_confident) on Foot: 0.0093
Accuracy Improvement (margin) on Foot: 0.0153
Accuracy Improvement (entropy) on Foot: 0.0202


évolution de l'accuracy en donnant jusqu'à 30% du training set avec différents batch_ratio. 1% initialement labellisé

In [136]:
# Paramètres fixes
labeled_ratio = 0.01  # 1% de données labellisées initialement
test_ratio = 0.2  # Proportion du dataset réservée au test
methods = ["random", "least_confident", "margin", "entropy"]
model_class = lambda: RandomForestClassifier()
datasets={"MNIST":(X_MNIST,y_MNIST)}

# Liste des batch ratios à tester
batch_ratios = [0.01,0.02,0.05,0.075,0.1,0.5]

# Stockage des résultats
accuracy_results = {}

for batch_ratio in batch_ratios:
    # Calculer le nombre d'itérations nécessaires pour atteindre 30%
    n_iterations = int(np.ceil((0.50 - labeled_ratio) / batch_ratio))
    
    print(f"Testing batch_ratio = {batch_ratio} with n_iterations = {n_iterations}")
    
    # Exécuter l'expérience
    accuracy_results[batch_ratio] = run_active_learning_experiment_datasets(
        datasets, labeled_ratio, test_ratio, n_iterations, batch_ratio, methods, model_class, models
    )





Traitement du dataset: MNIST
Taille totale du dataset MNIST: 100.00% (10000/10000)
Taille de l'ensemble de test: 20.00% (2000/10000)
Taille de l'ensemble de training: 80.00% (8000/10000)
Taille de l'ensemble labellisé dans le training set: 1.25% (100/8000)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (7900/8000)
Nb d'itérations: 49
Nb de données labellisées en plus à chaque itération: 1.00% (80/8000)


Testing batch_ratio = 0.01 with n_iterations = 49


random - Iteration 1: 2.25% (180/8000) samples labeled, Accuracy: 0.6970 on MNIST
random - Iteration 2: 3.25% (260/8000) samples labeled, Accuracy: 0.7565 on MNIST
random - Iteration 3: 4.25% (340/8000) samples labeled, Accuracy: 0.8210 on MNIST
random - Iteration 4: 5.25% (420/8000) samples labeled, Accuracy: 0.8395 on MNIST
random - Iteration 5: 6.25% (500/8000) samples labeled, Accuracy: 0.8560 on MNIST
random - Iteration 6: 7.25% (580/8000) samples labeled, Accuracy: 0.8585 on MNIST
random - Iteration 7: 8.25% (660/8000) samples labeled, Accuracy: 0.8815 on MNIST
random - Iteration 8: 9.25% (740/8000) samples labeled, Accuracy: 0.8790 on MNIST
random - Iteration 9: 10.25% (820/8000) samples labeled, Accuracy: 0.8805 on MNIST
random - Iteration 10: 11.25% (900/8000) samples labeled, Accuracy: 0.8940 on MNIST
random - Iteration 11: 12.25% (980/8000) samples labeled, Accuracy: 0.8985 on MNIST
random - Iteration 12: 13.25% (1060/8000) samples labeled, Accuracy: 0.8990 on MNIST
random -

Initial Accuracy on MNIST: 0.7010
Full Training Set Accuracy on MNIST: 0.9480
Accuracy Improvement (random) on MNIST: 0.2315
Accuracy Improvement (least_confident) on MNIST: 0.2520
Accuracy Improvement (margin) on MNIST: 0.2610
Accuracy Improvement (entropy) on MNIST: 0.2520

Traitement du dataset: MNIST
Taille totale du dataset MNIST: 100.00% (10000/10000)
Taille de l'ensemble de test: 20.00% (2000/10000)
Taille de l'ensemble de training: 80.00% (8000/10000)
Taille de l'ensemble labellisé dans le training set: 1.25% (100/8000)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (7900/8000)
Nb d'itérations: 25
Nb de données labellisées en plus à chaque itération: 2.00% (160/8000)


Testing batch_ratio = 0.02 with n_iterations = 25


...

random - Iteration 1: 11.25% (900/8000) samples labeled, Accuracy: 0.6830 on MNIST
random - Iteration 2: 21.25% (1700/8000) samples labeled, Accuracy: 0.8930 on MNIST
random - Iteration 3: 31.25% (2500/8000) samples labeled, Accuracy: 0.9155 on MNIST
random - Iteration 4: 41.25% (3300/8000) samples labeled, Accuracy: 0.9255 on MNIST
random - Iteration 5: 51.25% (4100/8000) samples labeled, Accuracy: 0.9310 on MNIST
Final Accuracy (random) on MNIST: 0.9310
least_confident - Iteration 1: 11.25% (900/8000) samples labeled, Accuracy: 0.6960 on MNIST
least_confident - Iteration 2: 21.25% (1700/8000) samples labeled, Accuracy: 0.8345 on MNIST
least_confident - Iteration 3: 31.25% (2500/8000) samples labeled, Accuracy: 0.9235 on MNIST
least_confident - Iteration 4: 41.25% (3300/8000) samples labeled, Accuracy: 0.9490 on MNIST
least_confident - Iteration 5: 51.25% (4100/8000) samples labeled, Accuracy: 0.9535 on MNIST
Final Accuracy (least_confident) on MNIST: 0.9535
margin - Iteration 1: 11.2

Initial Accuracy on MNIST: 0.6910
Full Training Set Accuracy on MNIST: 0.9500
Accuracy Improvement (random) on MNIST: 0.2400
Accuracy Improvement (least_confident) on MNIST: 0.2625
Accuracy Improvement (margin) on MNIST: 0.2660
Accuracy Improvement (entropy) on MNIST: 0.2565

Traitement du dataset: MNIST
Taille totale du dataset MNIST: 100.00% (10000/10000)
Taille de l'ensemble de test: 20.00% (2000/10000)
Taille de l'ensemble de training: 80.00% (8000/10000)
Taille de l'ensemble labellisé dans le training set: 1.25% (100/8000)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (7900/8000)
Nb d'itérations: 1
Nb de données labellisées en plus à chaque itération: 50.00% (4000/8000)


Testing batch_ratio = 0.5 with n_iterations = 1


random - Iteration 1: 51.25% (4100/8000) samples labeled, Accuracy: 0.6795 on MNIST
Final Accuracy (random) on MNIST: 0.6795
least_confident - Iteration 1: 51.25% (4100/8000) samples labeled, Accuracy: 0.6940 on MNIST
Final Accuracy (least_confident) on MNIST: 0.6940
margin - Iteration 1: 51.25% (4100/8000) samples labeled, Accuracy: 0.6965 on MNIST
Final Accuracy (margin) on MNIST: 0.6965
entropy - Iteration 1: 51.25% (4100/8000) samples labeled, Accuracy: 0.6650 on MNIST
Final Accuracy (entropy) on MNIST: 0.6650


Initial Accuracy on MNIST: 0.6890
Full Training Set Accuracy on MNIST: 0.9510
Accuracy Improvement (random) on MNIST: -0.0095
Accuracy Improvement (least_confident) on MNIST: 0.0050
Accuracy Improvement (margin) on MNIST: 0.0075
Accuracy Improvement (entropy) on MNIST: -0.0240


In [137]:
# Paramètres fixes
labeled_ratio = 0.01  # 1% de données labellisées initialement
test_ratio = 0.2  # Proportion du dataset réservée au test
methods = ["random", "least_confident", "margin", "entropy"]
model_class = lambda: RandomForestClassifier()
datasets={"PRS":(X_PRS,y_PRS)}

# Liste des batch ratios à tester
batch_ratios = [0.01,0.02,0.05,0.075,0.1,0.5]

# Stockage des résultats
accuracy_results = {}

for batch_ratio in batch_ratios:
    # Calculer le nombre d'itérations nécessaires pour atteindre 30%
    n_iterations = int(np.ceil((0.50 - labeled_ratio) / batch_ratio))
    
    print(f"Testing batch_ratio = {batch_ratio} with n_iterations = {n_iterations}")
    
    # Exécuter l'expérience
    accuracy_results[batch_ratio] = run_active_learning_experiment_datasets(
        datasets, labeled_ratio, test_ratio, n_iterations, batch_ratio, methods, model_class, models
    )





Traitement du dataset: PRS
Taille totale du dataset PRS: 100.00% (22250/22250)
Taille de l'ensemble de test: 20.00% (4450/22250)
Taille de l'ensemble de training: 80.00% (17800/22250)
Taille de l'ensemble labellisé dans le training set: 1.25% (222/17800)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (17578/17800)
Nb d'itérations: 49
Nb de données labellisées en plus à chaque itération: 1.00% (178/17800)


Testing batch_ratio = 0.01 with n_iterations = 49


random - Iteration 1: 2.25% (400/17800) samples labeled, Accuracy: 0.8578 on PRS
random - Iteration 2: 3.25% (578/17800) samples labeled, Accuracy: 0.8667 on PRS
random - Iteration 3: 4.25% (756/17800) samples labeled, Accuracy: 0.8719 on PRS
random - Iteration 4: 5.25% (934/17800) samples labeled, Accuracy: 0.8742 on PRS
random - Iteration 5: 6.25% (1112/17800) samples labeled, Accuracy: 0.8755 on PRS
random - Iteration 6: 7.25% (1290/17800) samples labeled, Accuracy: 0.8796 on PRS
random - Iteration 7: 8.25% (1468/17800) samples labeled, Accuracy: 0.8822 on PRS
random - Iteration 8: 9.25% (1646/17800) samples labeled, Accuracy: 0.8836 on PRS
random - Iteration 9: 10.25% (1824/17800) samples labeled, Accuracy: 0.8845 on PRS
random - Iteration 10: 11.25% (2002/17800) samples labeled, Accuracy: 0.8858 on PRS
random - Iteration 11: 12.25% (2180/17800) samples labeled, Accuracy: 0.8856 on PRS
random - Iteration 12: 13.25% (2358/17800) samples labeled, Accuracy: 0.8881 on PRS
random - Iter

Initial Accuracy on PRS: 0.8535
Full Training Set Accuracy on PRS: 0.8955
Accuracy Improvement (random) on PRS: 0.0387
Accuracy Improvement (least_confident) on PRS: 0.0422
Accuracy Improvement (margin) on PRS: 0.0427
Accuracy Improvement (entropy) on PRS: 0.0409

Traitement du dataset: PRS
Taille totale du dataset PRS: 100.00% (22250/22250)
Taille de l'ensemble de test: 20.00% (4450/22250)
Taille de l'ensemble de training: 80.00% (17800/22250)
Taille de l'ensemble labellisé dans le training set: 1.25% (222/17800)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (17578/17800)
Nb d'itérations: 25
Nb de données labellisées en plus à chaque itération: 2.00% (356/17800)


Testing batch_ratio = 0.02 with n_iterations = 25


...

random - Iteration 1: 11.25% (2002/17800) samples labeled, Accuracy: 0.8528 on PRS
random - Iteration 2: 21.25% (3782/17800) samples labeled, Accuracy: 0.8858 on PRS
random - Iteration 3: 31.25% (5562/17800) samples labeled, Accuracy: 0.8861 on PRS
random - Iteration 4: 41.25% (7342/17800) samples labeled, Accuracy: 0.8906 on PRS
random - Iteration 5: 51.25% (9122/17800) samples labeled, Accuracy: 0.8865 on PRS
Final Accuracy (random) on PRS: 0.8865
least_confident - Iteration 1: 11.25% (2002/17800) samples labeled, Accuracy: 0.8521 on PRS
least_confident - Iteration 2: 21.25% (3782/17800) samples labeled, Accuracy: 0.8849 on PRS
least_confident - Iteration 3: 31.25% (5562/17800) samples labeled, Accuracy: 0.8906 on PRS
least_confident - Iteration 4: 41.25% (7342/17800) samples labeled, Accuracy: 0.8953 on PRS
least_confident - Iteration 5: 51.25% (9122/17800) samples labeled, Accuracy: 0.8939 on PRS
Final Accuracy (least_confident) on PRS: 0.8939
margin - Iteration 1: 11.25% (2002/178

Initial Accuracy on PRS: 0.8542
Full Training Set Accuracy on PRS: 0.8989
Accuracy Improvement (random) on PRS: 0.0324
Accuracy Improvement (least_confident) on PRS: 0.0398
Accuracy Improvement (margin) on PRS: 0.0422
Accuracy Improvement (entropy) on PRS: 0.0387

Traitement du dataset: PRS
Taille totale du dataset PRS: 100.00% (22250/22250)
Taille de l'ensemble de test: 20.00% (4450/22250)
Taille de l'ensemble de training: 80.00% (17800/22250)
Taille de l'ensemble labellisé dans le training set: 1.25% (222/17800)
Taille de l'ensemble non-labellisé dans le training set: 98.75% (17578/17800)
Nb d'itérations: 1
Nb de données labellisées en plus à chaque itération: 50.00% (8900/17800)


Testing batch_ratio = 0.5 with n_iterations = 1


random - Iteration 1: 51.25% (9122/17800) samples labeled, Accuracy: 0.8573 on PRS
Final Accuracy (random) on PRS: 0.8573
least_confident - Iteration 1: 51.25% (9122/17800) samples labeled, Accuracy: 0.8528 on PRS
Final Accuracy (least_confident) on PRS: 0.8528
margin - Iteration 1: 51.25% (9122/17800) samples labeled, Accuracy: 0.8510 on PRS
Final Accuracy (margin) on PRS: 0.8510
entropy - Iteration 1: 51.25% (9122/17800) samples labeled, Accuracy: 0.8544 on PRS
Final Accuracy (entropy) on PRS: 0.8544


Initial Accuracy on PRS: 0.8584
Full Training Set Accuracy on PRS: 0.8960
Accuracy Improvement (random) on PRS: -0.0011
Accuracy Improvement (least_confident) on PRS: -0.0056
Accuracy Improvement (margin) on PRS: -0.0074
Accuracy Improvement (entropy) on PRS: -0.0040


In [138]:
# Paramètres fixes
labeled_ratio = 0.01  # 1% de données labellisées initialement
test_ratio = 0.2  # Proportion du dataset réservée au test
methods = ["random", "least_confident", "margin", "entropy"]
model_class = lambda: RandomForestClassifier()
datasets={"Foot":(X_foot,y_foot)}

# Liste des batch ratios à tester
batch_ratios = [0.01,0.02,0.05,0.075,0.1,0.5]

# Stockage des résultats
accuracy_results = {}

for batch_ratio in batch_ratios:
    # Calculer le nombre d'itérations nécessaires pour atteindre 30%
    n_iterations = int(np.ceil((0.50 - labeled_ratio) / batch_ratio))
    
    print(f"Testing batch_ratio = {batch_ratio} with n_iterations = {n_iterations}")
    
    # Exécuter l'expérience
    accuracy_results[batch_ratio] = run_active_learning_experiment_datasets(
        datasets, labeled_ratio, test_ratio, n_iterations, batch_ratio, methods, model_class, models
    )





Traitement du dataset: Foot
Taille totale du dataset Foot: 100.00% (9180/9180)
Taille de l'ensemble de test: 20.00% (1836/9180)
Taille de l'ensemble de training: 80.00% (7344/9180)
Taille de l'ensemble labellisé dans le training set: 1.24% (91/7344)
Taille de l'ensemble non-labellisé dans le training set: 98.76% (7253/7344)
Nb d'itérations: 49
Nb de données labellisées en plus à chaque itération: 0.99% (73/7344)


Testing batch_ratio = 0.01 with n_iterations = 49


random - Iteration 1: 2.23% (164/7344) samples labeled, Accuracy: 0.4265 on Foot
random - Iteration 2: 3.23% (237/7344) samples labeled, Accuracy: 0.5370 on Foot
random - Iteration 3: 4.22% (310/7344) samples labeled, Accuracy: 0.5866 on Foot
random - Iteration 4: 5.22% (383/7344) samples labeled, Accuracy: 0.6008 on Foot
random - Iteration 5: 6.21% (456/7344) samples labeled, Accuracy: 0.6373 on Foot
random - Iteration 6: 7.20% (529/7344) samples labeled, Accuracy: 0.6770 on Foot
random - Iteration 7: 8.20% (602/7344) samples labeled, Accuracy: 0.6961 on Foot
random - Iteration 8: 9.19% (675/7344) samples labeled, Accuracy: 0.7124 on Foot
random - Iteration 9: 10.19% (748/7344) samples labeled, Accuracy: 0.7293 on Foot
random - Iteration 10: 11.18% (821/7344) samples labeled, Accuracy: 0.7424 on Foot
random - Iteration 11: 12.17% (894/7344) samples labeled, Accuracy: 0.7495 on Foot
random - Iteration 12: 13.17% (967/7344) samples labeled, Accuracy: 0.7609 on Foot
random - Iteration 13

Initial Accuracy on Foot: 0.4581
Full Training Set Accuracy on Foot: 0.9363
Accuracy Improvement (random) on Foot: 0.4346
Accuracy Improvement (least_confident) on Foot: 0.4842
Accuracy Improvement (margin) on Foot: 0.4864
Accuracy Improvement (entropy) on Foot: 0.4695

Traitement du dataset: Foot
Taille totale du dataset Foot: 100.00% (9180/9180)
Taille de l'ensemble de test: 20.00% (1836/9180)
Taille de l'ensemble de training: 80.00% (7344/9180)
Taille de l'ensemble labellisé dans le training set: 1.24% (91/7344)
Taille de l'ensemble non-labellisé dans le training set: 98.76% (7253/7344)
Nb d'itérations: 25
Nb de données labellisées en plus à chaque itération: 1.99% (146/7344)


Testing batch_ratio = 0.02 with n_iterations = 25


...

random - Iteration 1: 11.23% (825/7344) samples labeled, Accuracy: 0.4352 on Foot
random - Iteration 2: 21.23% (1559/7344) samples labeled, Accuracy: 0.7511 on Foot
random - Iteration 3: 31.22% (2293/7344) samples labeled, Accuracy: 0.8235 on Foot
random - Iteration 4: 41.22% (3027/7344) samples labeled, Accuracy: 0.8649 on Foot
random - Iteration 5: 51.21% (3761/7344) samples labeled, Accuracy: 0.8753 on Foot
Final Accuracy (random) on Foot: 0.8753
least_confident - Iteration 1: 11.23% (825/7344) samples labeled, Accuracy: 0.4330 on Foot
least_confident - Iteration 2: 21.23% (1559/7344) samples labeled, Accuracy: 0.6547 on Foot
least_confident - Iteration 3: 31.22% (2293/7344) samples labeled, Accuracy: 0.7685 on Foot
least_confident - Iteration 4: 41.22% (3027/7344) samples labeled, Accuracy: 0.8584 on Foot
least_confident - Iteration 5: 51.21% (3761/7344) samples labeled, Accuracy: 0.8943 on Foot
Final Accuracy (least_confident) on Foot: 0.8943
margin - Iteration 1: 11.23% (825/7344

Initial Accuracy on Foot: 0.4357
Full Training Set Accuracy on Foot: 0.9368
Accuracy Improvement (random) on Foot: 0.4395
Accuracy Improvement (least_confident) on Foot: 0.4586
Accuracy Improvement (margin) on Foot: 0.4826
Accuracy Improvement (entropy) on Foot: 0.4308

Traitement du dataset: Foot
Taille totale du dataset Foot: 100.00% (9180/9180)
Taille de l'ensemble de test: 20.00% (1836/9180)
Taille de l'ensemble de training: 80.00% (7344/9180)
Taille de l'ensemble labellisé dans le training set: 1.24% (91/7344)
Taille de l'ensemble non-labellisé dans le training set: 98.76% (7253/7344)
Nb d'itérations: 1
Nb de données labellisées en plus à chaque itération: 50.00% (3672/7344)


Testing batch_ratio = 0.5 with n_iterations = 1


random - Iteration 1: 51.24% (3763/7344) samples labeled, Accuracy: 0.4265 on Foot
Final Accuracy (random) on Foot: 0.4265
least_confident - Iteration 1: 51.24% (3763/7344) samples labeled, Accuracy: 0.4330 on Foot
Final Accuracy (least_confident) on Foot: 0.4330
margin - Iteration 1: 51.24% (3763/7344) samples labeled, Accuracy: 0.4205 on Foot
Final Accuracy (margin) on Foot: 0.4205
entropy - Iteration 1: 51.24% (3763/7344) samples labeled, Accuracy: 0.4292 on Foot
Final Accuracy (entropy) on Foot: 0.4292


Initial Accuracy on Foot: 0.4368
Full Training Set Accuracy on Foot: 0.9363
Accuracy Improvement (random) on Foot: -0.0103
Accuracy Improvement (least_confident) on Foot: -0.0038
Accuracy Improvement (margin) on Foot: -0.0163
Accuracy Improvement (entropy) on Foot: -0.0076
